In [12]:
import pandas as pd
import pymongo

In [10]:
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies

## Reviso cuáles son todas las categorías, para quedarme con las que necesito.

In [22]:
cursor = db.companies.find({'category_code':{'$exists': 'true'}})
categories = list(map(lambda r:r['category_code'], list(cursor)))

category_codes = []
for i in categories:
    if i not in category_codes:
        category_codes.append(i)
    else:
        pass
print(category_codes)

['enterprise', 'web', 'software', 'news', 'network_hosting', 'games_video', 'social', 'music', 'mobile', 'search', 'advertising', 'messaging', 'photo_video', 'finance', 'security', 'ecommerce', 'hardware', 'travel', 'public_relations', 'other', 'real_estate', 'semiconductor', 'analytics', 'health', 'legal', 'sports', 'biotech', 'cleantech', 'education', 'consulting', 'transportation', None, 'hospitality', 'fashion', 'nanotech', 'nonprofit', 'automotive', 'design', 'manufacturing', 'government', 'local', 'medical']


## Creo un nuevo Json con las coordenadas en el formato adecuado, y con las compañías filtradas por número de empleados, año de fundación, que no hayan sido deadpooled y por las categorías que me convienen.

In [67]:
def toGeoJSON(lista):
    return {
            "type": "Point",
            "coordinates":[lista[1],lista[0]]
        }

def convertCompany(company):
    dictList = []
    for index in range(len(company['offices'])):
        for key in company['offices']:
            if key['latitude'] != None and key['longitude'] != None:
                fields = {
                "name": company["name"],
                "category_code": company["category_code"],
                "number_of_employees": company["number_of_employees"],
                "founded_year": company["founded_year"],
                "position": toGeoJSON([key['latitude'],key['longitude']])   
                }
                dictList.append(fields)
        return dictList

cursor = db.companies.find({ "$and": [ { "offices.latitude": {"$exists": True} },{ "offices.longitude": {"$exists": True} }, {"founded_year" : {"$gte": 2003} }, {'deadpooled_year': {'$type': 10} }, {'number_of_employees' : {"$gte": 50} } , { '$or': [ {'category_code': 'web'}, {'category_code':'software'}, {'category_code':'mobile'},{'category_code': 'analytics'}, {'category_code': 'games_video'}, {'category_code':'ecommerce'}, {'category_code':'enterprise'},{'category_code':'finance'} ] } ] })

geoList = list(map(lambda r: convertCompany(r),cursor))
geoCompanies = [element for lista in geoList for element in lista]

## DataFrame correspondiente al database recién creado.

In [72]:
df = pd.DataFrame(geoCompanies)
df = df.drop_duplicates(subset='name')
df.head()

,category_code,founded_year,name,number_of_employees,position
0,mobile,2005,Jajah,110,"{'type': 'Point', 'coordinates': [-122.0899512..."
1,games_video,2007,Livestream,120,"{'type': 'Point', 'coordinates': [-73.995625, ..."
2,games_video,2007,Ustream,250,"{'type': 'Point', 'coordinates': [-122.07948, ..."
3,enterprise,2004,KickApps,72,"{'type': 'Point', 'coordinates': [-73.9843328,..."
7,games_video,2005,RockYou,106,"{'type': 'Point', 'coordinates': [-122.206893,..."


## Guardo en formato json la nueva colección con las compañías que me interesan y con las coordenadas correctas.

In [73]:
df.to_json("comp_with_coords.json", orient="records", lines=True)

## Recorro la ubicación de cada compañía para ver cuántas empresas tiene a 1000 metros a la redonda, y luego verifico cuántas tienen más, para usar ese número más adelante para quedarme solo con las que más compañías tienen a su alrededor.

In [180]:
cursor = db.comp_coords.find()
maxCount = []
for company in cursor:

    cursor2 = db.comp_coords.find({
      "position": {
        "$near": {
          "$geometry": {
            "type": "Point",
            "coordinates": [company['position']['coordinates'][0], company['position']['coordinates'][1]]
          },
          "$minDistance": 0,
          "$maxDistance":1000
        }
      }
    })

    maxCount.append(cursor2.count())
    
list(filter(lambda x: x == max(maxCount), maxCount))

/home/carlosmd14/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


[18, 18]

## Al saber que las que más compañías tienen alrededor tienen 18 compañías, guardo en una lista solo aquellas que tienen 18 compañías alrededor.

In [178]:
cursor = db.comp_coords.find()
companies = []
for company in cursor:

    cursor2 = db.comp_coords.find({
      "position": {
        "$near": {
          "$geometry": {
            "type": "Point",
            "coordinates": [company['position']['coordinates'][0], company['position']['coordinates'][1]]
          },
          "$minDistance": 0,
          "$maxDistance":1000
        }
      }
    })
    
    if cursor2.count() == 18:
        companies.append(company)


/home/carlosmd14/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


## Y hago un pequeño DataFrame de esas dos compañías resultantes.

In [179]:
df = pd.DataFrame(companies)
df.head()

,_id,category_code,founded_year,name,number_of_employees,position
0,5c6aa73124490307beef1fe3,games_video,2006,Curse,58,"{'type': 'Point', 'coordinates': [-122.399972,..."
1,5c6aa73124490307beef2088,web,2008,Lolapps,65,"{'type': 'Point', 'coordinates': [-122.3999719..."


In [165]:
cursor = db.comp_coords.find({
  "position": {
    "$near": {
      "$geometry": {
        "type": "Point",
        "coordinates": [-74, 40.7]
      },
      "$minDistance": 0,
      "$maxDistance":1000
    }
  }
})

print(cursor.count())

df = pd.DataFrame(list(cursor))
df.tail()

1


/home/carlosmd14/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


,_id,category_code,founded_year,name,number_of_employees,position
0,5c6aa73124490307beef2068,mobile,2006,Airtag,50,"{'type': 'Point', 'coordinates': [-74.0082089,..."


In [138]:
df2 = pd.DataFrame.from_dict("comp_with_coords.json", orient='columns')



df2.head()

ValueError: DataFrame constructor not properly called!

In [ ]:
{ "offices.latitude": {"$exists": True} },{ "offices.longitude": {"$exists": True} }